# 5.16 Normalization
## 1. 대소문자 통합(소문자)
## 2. 구두점 처리 (I`d, I`m) => 기본적으로는 분리 tokenize
##   => 형태소분석기가 기본적으로 해줌
## 3. 불용어(stopwords) 처리

## TOKENIZATION => 영어(lowercase) => 구두점(in, re) => 불용어(stopwords, [사전] in koreand => 단어의 길이, 빈도 => filtering = 욕설

In [1]:
sentence = "I`d like to learn more something"

### 구두점 통합

In [2]:
from string import punctuation
punctuation 
# Finland`s -> finland. finlands 어떤식으로 변형시킬지
# '오늘'의 => 오늘 의, 오늘의 

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [3]:
import re

re.escape(punctuation) # 정규식표현으로 변환

'\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~'

In [4]:
re.sub("[{0}]".format(re.escape(punctuation)), "", sentence)

'Id like to learn more something'

In [5]:
pattern = re.compile("[{0}]".format(re.escape(punctuation)))
pattern

re.compile(r'[\!\"\#\$\%\&\\'\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\\\]\^_\`\{\|\}\~]',
re.UNICODE)

In [6]:
from nltk.tokenize import word_tokenize

print(pattern.sub("", sentence.lower()))
print(" ".join(word_tokenize(sentence.lower())))
print("`차이를 알 수 있다")

id like to learn more something
i ` d like to learn more something
`차이를 알 수 있다


In [7]:
sentence = "오늘은 '목'요일"
print(pattern.sub("", sentence.lower()))
print(" ".join(word_tokenize(sentence.lower())))

오늘은 목요일
오늘은 '목'요일


In [8]:
re.sub("\s{2,}", " ", "i  d  asdf  2342 ")

'i d asdf 2342 '

### Stopwords
#### 어근이 아닌애들

In [9]:
from nltk.corpus import stopwords
stopwords.fileids()

['arabic',
 'azerbaijani',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish',
 'turkish']

In [10]:
from nltk.corpus import stopwords 
stop = stopwords.open("english").read()
len(stop), print(stop)

i
me
my
myself
we
our
ours
ourselves
you
you're
you've
you'll
you'd
your
yours
yourself
yourselves
he
him
his
himself
she
she's
her
hers
herself
it
it's
its
itself
they
them
their
theirs
themselves
what
which
who
whom
this
that
that'll
these
those
am
is
are
was
were
be
been
being
have
has
had
having
do
does
did
doing
a
an
the
and
but
if
or
because
as
until
while
of
at
by
for
with
about
against
between
into
through
during
before
after
above
below
to
from
up
down
in
out
on
off
over
under
again
further
then
once
here
there
when
where
why
how
all
any
both
each
few
more
most
other
some
such
no
nor
not
only
own
same
so
than
too
very
s
t
can
will
just
don
don't
should
should've
now
d
ll
m
o
re
ve
y
ain
aren
aren't
couldn
couldn't
didn
didn't
doesn
doesn't
hadn
hadn't
hasn
hasn't
haven
haven't
isn
isn't
ma
mightn
mightn't
mustn
mustn't
needn
needn't
shan
shan't
shouldn
shouldn't
wasn
wasn't
weren
weren't
won
won't
wouldn
wouldn't



(936, None)

In [11]:
sentence = "I love you"
sentence = "Beautiful is better than ugly."
# for _ in sentence.lower().split():
#     if _ in stop:
#         print("Skipped")
#     else:
#         print(_)
# 한글은 1음절단위라 stopwords는 ngram으로 처리해야한다
for _ in word_tokenize(sentence.lower()): # 1번
    if pattern.sub("", _) in stop: # 2번 with stopwords
        print("Skipped")
    else:
        print(_)

beautiful
Skipped
better
Skipped
ugly
Skipped


### stopwords 처리하면 단어의수가 엄청 줄어든다. 19만개 -> 6만개
### 하지만 중요한 말이 포함될 수 도 있다. ex) "to be or not to be"

In [12]:
from nltk.corpus import gutenberg

corpus = gutenberg.open("austen-emma.txt").read()

len(word_tokenize(corpus))

191785

In [13]:
words = list()
for _ in word_tokenize(corpus.lower()):
    if pattern.sub("", _) not in stop:
        words.append(_)
len(words)

69791

### 한국어 불용어

In [14]:
korstop =["은", "는", "이", "기", "을", "를", "께서", "에게", "을", "를", "고", "어", "."]

sentence = "어머니 는 짜장면 이 싫다 고 하셨 어."

word_tokenize(sentence)

['어머니', '는', '짜장면', '이', '싫다', '고', '하셨', '어', '.']

In [15]:
[_ for _ in word_tokenize(sentence) if _ not in korstop]

['어머니', '짜장면', '싫다', '하셨']

### 길이 정규화 => 특정 길이(너무 짧거나 너무 길거나)

In [41]:
sentence = "I'd like to learn more something. I'd like to learn. I'd like"
for _ in pattern.sub("", sentence.lower()).split():
    if 2 < len(_) < 6:
        print(_)

like
learn
more
like
learn
like


In [42]:
minimum = 2
maximum = 6
pattern2 = re.compile(r"\b\w{%d,%d}\b" % (minimum, maximum))
pattern2.findall(sentence)

['like', 'to', 'learn', 'more', 'like', 'to', 'learn', 'like']

### 빈도로 정규화

In [43]:
from nltk import Text
obj = Text(word_tokenize(pattern.sub("",sentence.lower())))
for _ in obj.vocab():
    if 1 < obj.vocab().get(_) < 3:
        print(_, obj.vocab().get(_))

to 2
learn 2


### 말뭉치로 정규화 해보기
### N : 전체단어개수
### B : 유니크한 단어 개수

In [32]:
original = Text(word_tokenize(corpus))
print(original.vocab().N(), original.vocab().B(), original.vocab().most_common(50))

lowercase = Text(word_tokenize(corpus.lower()))
print(lowercase.vocab().N(), lowercase.vocab().B())

punct1 = Text(word_tokenize(pattern.sub("", corpus.lower())))
print(punct1.vocab().N(), punct1.vocab().B())

punct2 = Text(word_tokenize(pattern.sub(" ", corpus.lower())))
print(punct2.vocab().N(), punct2.vocab().B())

stops = Text([_ for _ in word_tokenize(pattern.sub(" ", corpus.lower())) if _ not in stop])
print(stops.vocab().N(), stops.vocab().B())

191785 8406 [(',', 12016), ('.', 6355), ('to', 5125), ('the', 4844), ('and', 4653), ('of', 4272), ('I', 3177), ('--', 3100), ('a', 3001), ("''", 2452), ('was', 2383), ('her', 2360), (';', 2353), ('not', 2242), ('in', 2103), ('it', 2103), ('be', 1965), ('she', 1774), ('``', 1735), ('that', 1729), ('you', 1664), ('had', 1605), ('as', 1387), ('he', 1365), ('for', 1320), ('have', 1301), ('is', 1221), ('with', 1185), ('very', 1151), ('but', 1148), ('Mr.', 1091), ('his', 1084), ('!', 1063), ('at', 996), ('so', 918), ("'s", 866), ('Emma', 855), ('all', 831), ('could', 824), ('would', 813), ('been', 755), ('him', 748), ('on', 674), ('Mrs.', 668), ('any', 651), ('?', 621), ('my', 619), ('no', 616), ('Miss', 592), ('were', 590)]
191781 7944
158270 9311
162122 7102
70168 6933


### 글자길이 > 3

In [49]:
advanced = Text([_ for _ in word_tokenize(pattern.sub(" ", corpus.lower())) if _ not in stop and len(_) > 3])

print(advanced.vocab().N(), advanced.vocab().B())

63814 6763


In [50]:
advanced = Text([_ for _ in word_tokenize(pattern.sub(" ", corpus.lower())) if _ not in stop and \
                 re.search(r"\b\w{4,}\b", _)])
print(advanced.vocab().N(), advanced.vocab().B())

63814 6763


### 빈도 > 10

In [52]:
advanced = Text([_ for _ in word_tokenize(pattern.sub(" ", corpus.lower())) if _ not in stop and len(_) > 3])
result = list()
for _ in advanced:
    if advanced.vocab().get(_) > 10:
        result.append(_)
        
print(Text(result).vocab().N(), Text(result).vocab().B())

48961 1111


In [56]:
freq = [(_, advanced.count(_)) for _ in advanced.vocab()
            if advanced.vocab().get(_) > 10]
print(sum([_[1] for _ in freq]), len(freq))

48961 1111


In [60]:
type(Text(result).vocab())
Text(result).vocab().freq("emma") # "emma"가 차지하는 부분을 알려줌 바율로 자르는게 빈도수로 자르는것보다 안정적

0.01766712281203407

# 한글 불용어와 비속어 처리

In [76]:
# => 사전정보 lexicon resource (못구함 돈이기때문에)
stop = ["시발"]
sentence = "시발 시발짜증나 시발!ㅁㄴㅇㅇㄻㄴㅇ234 시1발 시!발"

result = list()
for _ in sentence.split():
#     if _ not in stop:
    if not re.search(stop[0], _):
        result.append(_)
    else:
        result.append("*" * len(_))

" ".join(result)

'** ***** ************* 시1발 시!발'

### bigram을 사용해서 하기

In [77]:
# bigram 구현
def umjeol(text, n=2):
    ngram = list()
    for i in range(len(text)-(n-1)):
        ngram.append("".join(text[i:i+n]))
    return ngram

In [79]:
result = list()
for _ in sentence.split():
    flag = False
    for ngram in umjeol(_):
        if ngram in stop:
            flag = True

    if not flag:
        result.append(_)
    else:
        result.append("*" * len(_))
" ".join(result)

'** ***** ************* 시1발 시!발'

In [84]:
sentence = "시발 시발짜증나 시발!ㅁㄴㅇㅇㄻㄴㅇ234 시1발 시!발"

result = list()
for _ in sentence.split():
    if not re.search(stop[0], re.sub(r"\B[0-9!]+\B", "", _)):
        result.append(_)
    else:
        result.append("*" * len(_))
        
" ".join(result)

'** ***** ************* *** 시!발'

### Byte pair encoding
#### stopwords => list(dictionary) 힘들가
### BPE => 패턴(쌍)찾아서 적용할 수 있다

In [137]:
def splitTerm(term):
    result = list()
    for token in term.split():
        result.append(" ".join(["</w>"] + list(term) + ["</w>"]))
    return " _ ".join(result)
#     print(" ".join(list(term) + ["</w>"])) # /w는 단어의 경계

In [148]:
import re

def mergeNgram(maxKey, data):
    newData = dict()
    
    for term, freq in data.items():
        newKey = re.sub(maxKey, maxKey.replace(' ',''), term)
        newData[newKey] = freq
    
    return newData

In [149]:
from collections import defaultdict

def ngram(data, n=2):
    result = defaultdict(int)
    for term, freq in data.items():
        tokens = term.split()
        for i in range(len(tokens)-(n-1)):
#             key = " ".join(tokens[i:i+n])
#             if key in result.keys():
#                 result[key] += freq
#             else:
#                 result[key] = freq
            result[" ".join(tokens[i:i+n])] += freq
            
    return result

In [150]:
data = {
    splitTerm("시발"):2,
    splitTerm("시1발"):2,
    splitTerm("시~@1발"):6,
    splitTerm("시123123135발"):3
}

In [151]:
bigram = ngram(data)
bigram

defaultdict(int,
            {'</w> 시': 13,
             '시 발': 2,
             '발 </w>': 13,
             '시 1': 5,
             '1 발': 8,
             '시 ~': 6,
             '~ @': 6,
             '@ 1': 6,
             '1 2': 6,
             '2 3': 6,
             '3 1': 6,
             '1 3': 3,
             '3 5': 3,
             '5 발': 3})

In [152]:
for _ in range(2):
    bigram = ngram(data)
    maxKey = max(bigram, key=bigram.get)
    data = mergeNgram(maxKey, data)
#     print(data)
data

{'</w>시 발</w>': 2,
 '</w>시 1 발</w>': 2,
 '</w>시 ~ @ 1 발</w>': 6,
 '</w>시 1 2 3 1 2 3 1 3 5 발</w>': 3}

In [153]:
pattern = defaultdict(int)
for _ in data:
    for token in set(_.split()):
        pattern[token] += data[_]

sorted(pattern.items(), key=lambda x:x[1], reverse=True)

[('발</w>', 13),
 ('</w>시', 13),
 ('1', 11),
 ('@', 6),
 ('~', 6),
 ('2', 3),
 ('3', 3),
 ('5', 3)]

In [109]:
stopRE = re.compile(r"{0}.*{1}".format(pattern[0][0], pattern[1][0]))

TypeError: 'int' object is not subscriptable

In [ ]:
for _ in ["</w>" + _ + "</w>" for _ in sentence.split()]:
    if stopRE.search(_):
        print(_)
    else:
        print("Skipped")


In [98]:
for _ in range(2):
    bigram = ngram(data)
    maxKey = max(bigram, key=bigram.get)
    data = mergeNgram(maxKey, data)
#     print(data)
data

TypeError: unsupported operand type(s) for +=: 'int' and 'list'

# 영어 형태소 분석기 테스트 feat. Stanford

In [125]:
from nltk.help import brown_tagset

brown_tagset("N")

NN: noun, singular, common
    failure burden court fire appointment awarding compensation Mayor
    interim committee fact effect airport management surveillance jail
    doctor intern extern night weekend duty legislation Tax Office ...
NN$: noun, singular, common, genitive
    season's world's player's night's chapter's golf's football's
    baseball's club's U.'s coach's bride's bridegroom's board's county's
    firm's company's superintendent's mob's Navy's ...
NN+BEZ: noun, singular, common + verb 'to be', present tense, 3rd person singular
    water's camera's sky's kid's Pa's heat's throat's father's money's
    undersecretary's granite's level's wife's fat's Knife's fire's name's
    hell's leg's sun's roulette's cane's guy's kind's baseball's ...
NN+HVD: noun, singular, common + verb 'to have', past tense
    Pa'd
NN+HVZ: noun, singular, common + verb 'to have', present tense, 3rd person singular
    guy's Knife's boat's summer's rain's company's
NN+IN: noun, singular, common

### stanford  tagger가 제일 유명함 model을 넣어야돼서 따로 다운로드 해야함

In [132]:
from nltk.tag.stanford import StanfordPOSTagger
MODEL = "./stanford-postagger-full-2018-10-16/models/english-bidirectional-distsim.tagger"
PARSER = "./stanford-postagger-full-2018-10-16/stanford-postagger-3.9.2.jar"
pos = StanfordPOSTagger(MODEL, PARSER)

In [136]:
from nltk.tokenize import sent_tokenize

pos.tag(word_tokenize(sent_tokenize(corpus)[0]))
# pos.tag_sents()

[('[', 'NNP'),
 ('Emma', 'NNP'),
 ('by', 'IN'),
 ('Jane', 'NNP'),
 ('Austen', 'NNP'),
 ('1816', 'CD'),
 (']', 'NNP'),
 ('VOLUME', 'NNP'),
 ('I', 'NNP'),
 ('CHAPTER', 'NNP'),
 ('I', 'NNP'),
 ('Emma', 'NNP'),
 ('Woodhouse', 'NNP'),
 (',', ','),
 ('handsome', 'JJ'),
 (',', ','),
 ('clever', 'JJ'),
 (',', ','),
 ('and', 'CC'),
 ('rich', 'JJ'),
 (',', ','),
 ('with', 'IN'),
 ('a', 'DT'),
 ('comfortable', 'JJ'),
 ('home', 'NN'),
 ('and', 'CC'),
 ('happy', 'JJ'),
 ('disposition', 'NN'),
 (',', ','),
 ('seemed', 'VBD'),
 ('to', 'TO'),
 ('unite', 'VB'),
 ('some', 'DT'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('best', 'JJS'),
 ('blessings', 'NNS'),
 ('of', 'IN'),
 ('existence', 'NN'),
 (';', ':'),
 ('and', 'CC'),
 ('had', 'VBD'),
 ('lived', 'VBN'),
 ('nearly', 'RB'),
 ('twenty-one', 'CD'),
 ('years', 'NNS'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('world', 'NN'),
 ('with', 'IN'),
 ('very', 'RB'),
 ('little', 'JJ'),
 ('to', 'TO'),
 ('distress', 'NN'),
 ('or', 'CC'),
 ('vex', 'VB'),
 ('her', 'PRP'),
 ('.', '.')

In [129]:
from konlpy.tag import Kkma
Kkma().tagset

{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC